In [1]:
from database import *
from backtest import *
import json

/opt/anaconda3/lib/python3.8/site-packages/shioaji/__init__.py:20: Warning: Mac OS currently don't provide ca module all function about placing order is not working.
  warnings.warn("Mac OS currently don't provide ca module all function about placing order is not working.", Warning)


In [2]:
with open('config.json') as f:
  config = json.load(f)

api = sj.Shioaji()
api.login(
    person_id=config['account'], 
    passwd=config['password'], 
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)

connection = sqlite3.connect('data.db')

# api.activate_ca(
#     ca_path="./Sinopac.pfx",
#     ca_passwd="YOUR_CA_PASSWORD",
#     person_id="Person of this Ca",
# )

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.


In [3]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions = {}

workers = {}
stop_threads = False

@api.quote.on_quote
def quote_callback(topic: str, quote: dict):
    
    code = topic.split('/')[-1]
    
    ts = pd.to_datetime(quote.get('Date') + ' ' + quote.get('Time'))
    
    if ts.hour < 9:
        return 
    
    close = quote.get('Close')[0]
    volume = quote.get('Volume')[0]
    vol_sum = quote.get('VolSum')[0]
    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum
    
    
def place_cb(stat, msg):
    
    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])

        order_qty[code] -= qty
        
        if order_qty[code] == 0:
            open_orders[code] = None
        
        position = positions.get(code, 0)
        
        if msg['action'] == 'Sell':
            qty = qty * -1
            
        position += qty
        
        positions[code] = position 

api.set_order_callback(place_cb)

In [4]:
import time

def tbot(code):

    print('=== 啟動 tbot #{} ==='.format(code))
    
    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    if date not in tw_calendar.opens:
        print('今日非交易日')
        return 

    ticks[code] = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    ticks[code] = ticks[code][['close', 'volume']]
    kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='5Min'))
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()
    entry_price = -1
    open_price = -1
    api.quote.subscribe(api.Contracts.Stocks[code], quote_type='tick')
    
    while True:
        
        if stop_threads:
            break
            
        time.sleep(1)
        
        current_time = datetime.datetime.now()
        
        if current_time.second == 0:

            kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='5Min'))
            kbars[code] = kbars[code][date:]
            
            str_current_time = (current_time - datetime.timedelta(minutes=5)).strftime('%Y-%m-%d %H:%M:%S')
            str_prev_time = (current_time - datetime.timedelta(minutes=10)).strftime('%Y-%m-%d %H:%M:%S')
            str_open_time = current_time.replace(hour=9, minute=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
            try:
                current_price = kbars[code].loc[str_current_time]['close']
                prev_price = kbars[code].loc[str_prev_time]['close']
                if open_price == -1:
                    open_price = kbars[code].loc[str_open_time]['open']
                    # 開盤太高，不沖
                    if open_price * 1.04 <= kbars[code].loc[str_open_time]['close']:
                        break
            except:
                continue
 
            open_order = open_orders.get(code, None)
            
            if open_order:
                
                api.update_status(api.stock_account)
                
                if open_order.status.status in ['PendingSubmit', 'Submitted', 'Filling']:
                    
                    api.cancel_order(open_order)
                    
                    open_orders[code] = None
                    order_qty[code] = 0
                    
                    if open_order.status.status != 'Filling':
                        break
                    
            position = positions.get(code, 0)
            
            if (
                current_time < current_time.replace(hour=10, minute=30, second=0) and
                kbars[code].loc[str_prev_time]['cci'] < -350 and
                kbars[code].loc[str_prev_time]['slowk'] < kbars[code].loc[str_prev_time]['slowd'] and
                kbars[code].loc[str_current_time]['slowk'] > kbars[code].loc[str_current_time]['slowd'] and  
                position == 0
            ):
                if current_price < 20:
                    qty = 20
                    positions[code] = qty
                else:
                    qty = 2
                    positions[code] = qty
                    
#                     contract = api.Contracts.Stocks[code]
#                     order = api.Order(
#                         price=current_price,
#                         quantity=qty,
#                         action="Buy",
#                         price_type="LMT",
#                         order_type="ROD",
#                         account=api.stock_account
#                     )
                    
#                     trade = api.place_order(contract, order)

#                     open_orders[code] = trade
#                     order_qty[code] = qty

                entry_price = current_price
                print('[{}] buy {} at {}'.format(str_current_time, code, current_price))

                
            elif (
                    (current_price > open_price * 1.095 and position != 0) or
                    (current_price >= entry_price * 1.02 and kbars[code].loc[str_current_time]['cci'] <= 130 and position != 0) or
                    (kbars[code].loc[str_current_time]['cci'] > 130 and position != 0)
                ):
                
                    positions[code] = 0
                        
#                         contract = api.Contracts.Stocks[code]
#                         order = api.Order(
#                             price=current_price,
#                             quantity=position,
#                             action="Sell",
#                             price_type="LMT",
#                             order_type="ROD",
#                             account=api.stock_account
#                         )
                        
#                         trade = api.place_order(contract, order)

#                         open_orders[code] = trade
#                         order_qty[code] = position
                        
                    print('[{}] sell {} at {}'.format(str_current_time, code, current_price))
                    break
                    
            elif (
                current_time >= current_time.replace(hour=13, minute=15, second=0)

            ):
                if position != 0:
                    
                    positions[code] = 0
                    
#                     contract = api.Contracts.Stocks[code]
#                     order = api.Order(
#                         price=current_price,
#                         quantity=position,
#                         action="Sell",
#                         price_type="LMT",
#                         order_type="ROD",
#                         account=api.stock_account
#                     )

#                     trade = api.place_order(contract, order)

#                     open_orders[code] = trade
#                     order_qty[code] = position

                    print('[{}] sell {} at {}'.format(str_current_time, code, current_price))
                    
                break 
                
    api.quote.unsubscribe(api.Contracts.Stocks[code], quote_type='tick')
    
    print('=== 關閉 tbot #{} ==='.format(code))


In [5]:
import threading

def run():
    
    date = pd.to_datetime(datetime.datetime.now().date())
    
    codes = get_stocks(date, connection)    
    day_trading_codes = [code for code in codes if get_stock(code, connection, api)[0].iloc[0]['day_trade'] == 'Yes']

    print('今日當沖標的：{}'.format(day_trading_codes))
    
    for code in day_trading_codes:
        
        worker = threading.Thread(name=code, target=tbot, args=(code,))
        worker.start()
        workers[code] = worker

In [6]:
update_historial_data('2021/3/25', '2021/3/25', connection, api)

<SecurityType.Future: 'FUT'> fetch done.
正在更新每日收盤行情...
<SecurityType.Stock: 'STK'> fetch done.
<SecurityType.Option: 'OPT'> fetch done.
20210325 找不到資料
正在篩選每日當沖標的...
正在更新股票資訊...
正在更新逐筆成交資料...
股市資料庫更新完成


In [7]:
run()

今日當沖標的：['1524', '5521', '3596', '2455']
=== 啟動 tbot #1524 ===
=== 啟動 tbot #5521 ===
=== 啟動 tbot #3596 ===
=== 啟動 tbot #2455 ===


In [8]:
workers

{'1524': <Thread(1524, started 123145526837248)>,
 '5521': <Thread(5521, started 123145543626752)>,
 '3596': <Thread(3596, started 123145560416256)>,
 '2455': <Thread(2455, started 123145577205760)>}

In [11]:
# ticks
kbars
# volume_today

{'1524':                       open  close   high    low  volume         slowk  \
 ts                                                                      
 2021-03-25 09:00:00  11.40  11.35  11.45  11.20   559.0  5.833333e+01   
 2021-03-25 09:05:00  11.35  11.35  11.35  11.30    98.0  6.666667e+01   
 2021-03-25 09:10:00  11.35  11.35  11.35  11.30    36.0  7.500000e+01   
 2021-03-25 09:15:00  11.35  11.30  11.35  11.25   116.0  7.083333e+01   
 2021-03-25 09:20:00  11.30  11.25  11.30  11.25    84.0  6.250000e+01   
 2021-03-25 09:25:00  11.20  11.25  11.25  11.20    18.0  5.178571e+01   
 2021-03-25 09:30:00  11.25  11.25  11.30  11.25    39.0  4.523810e+01   
 2021-03-25 09:35:00  11.25  11.15  11.25  11.15   138.0  3.333333e+01   
 2021-03-25 09:40:00  11.15  11.15  11.25  11.15    37.0  1.904762e+01   
 2021-03-25 09:45:00  11.15  11.20  11.20  11.15    10.0  1.309524e+01   
 2021-03-25 09:50:00  11.25  11.25  11.25  11.25    21.0  2.500000e+01   
 2021-03-25 09:55:00  11.25  1

In [12]:
stop_threads = True

=== 關閉 tbot #5521 ===
Response Code: 200 | Event Code: 16 | Info: MKT/*/TSE/5521 | Event: Subscribe or Unsubscribe ok
=== 關閉 tbot #2455 ====== 關閉 tbot #3596 ====== 關閉 tbot #1524 ===


Response Code: 200 | Event Code: 16 | Info: MKT/*/TSE/2455 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: MKT/*/TSE/3596 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: MKT/*/TSE/1524 | Event: Subscribe or Unsubscribe ok
